<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/continuous_outcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Fragility Metrics Toolkit: Continuous Outcomes (Welch t-test)
# 20-NOV-2025
# Fully aligned with FRAGILITY_METRICS.md v9.5 §3.7 CFQ/CFS and §4 MeCI
#
# Input: m1, sd1, n1  (group 1), m2, sd2, n2  (group 2)
# Output: p (Welch t p-value), fr (CFS/CFQ), nb (MeCI)
#
# Formulas (exact v9.5):
# - CFS  = ||T| – t*|   (SE-unit distance to the α=0.05 boundary)
# - CFQ  = CFS / (1 + CFS)                  → fragility quotient 0–1
# - MeCI = D / (1 + D)   where D = |m1-m2| / SE_diff   → robustness 0–1
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit [Software]. Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

import numpy as np
from scipy.stats import t

ALPHA = 0.05

# ---------- Core calculations ----------
def welch_t_test(m1, sd1, n1, m2, sd2, n2):
    se1 = sd1 / np.sqrt(n1)
    se2 = sd2 / np.sqrt(n2)
    se_diff = np.sqrt(se1**2 + se2**2)
    t_stat = (m1 - m2) / se_diff
    df = (se1**2 + se2**2)**2 / (se1**4 / (n1 - 1) + se2**4 / (n2 - 1))
    p_val = 2 * t.sf(np.abs(t_stat), df)
    return t_stat, p_val, se_diff, df

def compute_cfs_cfq(t_stat, df, alpha=ALPHA):
    t_crit = t.ppf(1 - alpha/2, df)
    cfs = abs(t_stat) - t_crit
    cfq = cfs / (1 + cfs) if cfs >= 0 else 0.0   # CFS can be negative only if already nonsig → fragility=0
    return cfs, cfq

def compute_meci(m1, m2, se_diff):
    d = abs(m1 - m2) / se_diff
    return d / (1 + d)

# ---------- High-level ----------
def continuous_outcomes(m1, sd1, n1, m2, sd2, n2):
    if n1 < 2 or n2 < 2:
        raise ValueError("Sample sizes must be ≥2")
    if sd1 <= 0 or sd2 <= 0:
        raise ValueError("Standard deviations must be positive")

    t_stat, p_val, se_diff, df = welch_t_test(m1, sd1, n1, m2, sd2, n2)
    cfs, cfq = compute_cfs_cfq(t_stat, df)
    meci = compute_meci(m1, m2, se_diff)

    return {
        "m1": m1, "sd1": sd1, "n1": n1,
        "m2": m2, "sd2": sd2, "n2": n2,
        "t_stat": t_stat,
        "df": df,
        "p": p_val,
        "fr": {"CFS": cfs, "CFQ": cfq},
        "nb": {"MeCI": meci}
    }

# ---------- Narrative ----------
def generate_narrative(res):
    diff = abs(res["m1"] - res["m2"])
    se = (res["sd1"]**2 / res["n1"] + res["sd2"]**2 / res["n2"])**0.5

    lines = [
        f"Group 1: mean = {res['m1']:.4f}, sd = {res['sd1']:.4f}, n = {res['n1']}",
        f"Group 2: mean = {res['m2']:.4f}, sd = {res['sd2']:.4f}, n = {res['n2']}",
        f"Mean difference = {diff:.4f} (SE = {se:.4f})",
        f"Welch t = {res['t_stat']:.4f}, df ≈ {res['df']:.1f}, p = {res['p']:.6f}",
    ]

    # Fragility
    if res["fr"]["CFS"] <= 0:
        frag = "The result is already non-significant – fragility = 0 (CFQ = 0.000000)"
    else:
        stability = (
            "extremely fragile" if res["fr"]["CFQ"] < 0.01 else
            "very fragile" if res["fr"]["CFQ"] < 0.05 else
            "fragile" if res["fr"]["CFQ"] < 0.10 else
            "moderately stable" if res["fr"]["CFQ"] < 0.25 else
            "very stable"
        )
        frag = f"CFS = {res['fr']['CFS']:.4f} SE units to boundary → CFQ = {res['fr']['CFQ']:.6f} ({stability})"

    # Robustness
    separation = (
        "at neutrality boundary" if res["nb"]["MeCI"] < 0.05 else
        "near neutrality" if res["nb"]["MeCI"] < 0.10 else
        "moderately separated" if res["nb"]["MeCI"] < 0.25 else
        "clearly separated" if res["nb"]["MeCI"] < 0.50 else
        "far from neutrality"
    )
    rob = f"MeCI = {res['nb']['MeCI']:.6f} → {separation}"

    lines += ["", frag, rob, ""]
    if res["p"] <= ALPHA:
        lines.append("Claim of difference is currently supported and is " +
                     ("highly stable and robust." if res["fr"]["CFQ"] > 0.25 and res["nb"]["MeCI"] > 0.5 else
                      "fragile and/or close to neutrality."))
    else:
        lines.append("Claim of difference is currently NOT supported.")
    return "\n".join(lines)

# ---------- CLI ----------
def main():
    print("Continuous Outcomes Calculator (Welch t) – v9.5 compliant\n")
    print("Group 1")
    m1 = float(input("Mean: "))
    sd1 = float(input("SD: "))
    n1 = int(input("n: "))
    print("Group 2")
    m2 = float(input("Mean: "))
    sd2 = float(input("SD: "))
    n2 = int(input("n: "))

    res = continuous_outcomes(m1, sd1, n1, m2, sd2, n2)

    print("\n================ p–fr–nb ================")
    print(f"p (Welch t) = {res['p']:.6f}")
    print(f"CFS = {res['fr']['CFS']:.4f}")
    print(f"CFQ = {res['fr']['CFQ']:.6f}")
    print(f"MeCI = {res['nb']['MeCI']:.6f}")
    print("=========================================\n")

    print("Interpretation:")
    print(generate_narrative(res))

if __name__ == "__main__":
    main()

Failed to download RCW_Title_01.pdf: 404 Client Error: Not Found for url: https://leg.wa.gov/CodeReviser/documents/rcw/RCW_Title_01.pdf
Failed to download RCW_Title_02.pdf: 404 Client Error: Not Found for url: https://leg.wa.gov/CodeReviser/documents/rcw/RCW_Title_02.pdf
Failed to download RCW_Title_03.pdf: 404 Client Error: Not Found for url: https://leg.wa.gov/CodeReviser/documents/rcw/RCW_Title_03.pdf
Failed to download RCW_Title_04.pdf: 404 Client Error: Not Found for url: https://leg.wa.gov/CodeReviser/documents/rcw/RCW_Title_04.pdf
Failed to download RCW_Title_05.pdf: 404 Client Error: Not Found for url: https://leg.wa.gov/CodeReviser/documents/rcw/RCW_Title_05.pdf
Failed to download RCW_Title_06.pdf: 404 Client Error: Not Found for url: https://leg.wa.gov/CodeReviser/documents/rcw/RCW_Title_06.pdf
Failed to download RCW_Title_07.pdf: 404 Client Error: Not Found for url: https://leg.wa.gov/CodeReviser/documents/rcw/RCW_Title_07.pdf
Failed to download RCW_Title_08.pdf: 404 Client 